In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!wget https://www.dropbox.com/s/flh1fjynqvdsj4p/lexvec.commoncrawl.300d.W.pos.vectors.gz?dl=1
!wget https://www.dropbox.com/s/kguufyc2xcdi8yk/lexvec.enwiki%2Bnewscrawl.300d.W.pos.vectors.gz?dl=1

--2020-03-06 00:16:49--  https://www.dropbox.com/s/flh1fjynqvdsj4p/lexvec.commoncrawl.300d.W.pos.vectors.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/flh1fjynqvdsj4p/lexvec.commoncrawl.300d.W.pos.vectors.gz [following]
--2020-03-06 00:16:49--  https://www.dropbox.com/s/dl/flh1fjynqvdsj4p/lexvec.commoncrawl.300d.W.pos.vectors.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3b5d613e628bedc401d2ac7b8d.dl.dropboxusercontent.com/cd/0/get/AzUmeqS2C8X1JSBVxx4mUgstak0YeZmTg0uRv3761_XgCj9kMeQ1MXQTUiCux76y2whqBEmj5sBuxBQN5kpLAPKBu5b5nKUleOhwnuRK_8KwuA/file?dl=1# [following]
--2020-03-06 00:16:49--  https://uc3b5d613e628bedc401d2ac7b8d.dl.dropboxusercontent.com/cd/0/get/AzUmeqS2C8X1JSBVxx4mUgstak0YeZmTg0uRv3761_XgCj9kM

In [0]:
!gunzip lexvec_commoncrawl.gz
!gunzip lexvec_enwiki.gz

In [0]:
import io
import _pickle as pickle
import os
import numpy as np
def read_simlex():
    return pd.read_table('drive/My Drive/SimLex999.txt', usecols=['word1', 'word2', 'SimLex999']).rename(
        columns={'SimLex999': 'gt_sim'}) 

def extract_simlex_vocabulary(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as doc:
            simlex_vocab = pickle.load(doc)
    else:
        simlex_pairs = read_simlex()
        simlex_vocab = list()
        simlex_vocab.extend(simlex_pairs['word1'].get_values())
        simlex_vocab.extend(simlex_pairs['word1'].get_values())
        simlex_vocab = list(set([w.lower() for w in simlex_vocab]))
        with open(file_path, 'wb') as doc:
            pickle.dump(simlex_vocab, doc)
    return simlex_vocab

def read_wordsim():
    return pd.read_table('data/WordSim353.csv', sep=',').rename(columns={'Human (mean)': 'gt_sim'})

def extract_wordsim_vocabulary(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as doc:
            wordsim_vocab = pickle.load(doc)
    else:
        wordsim_pairs = read_wordsim()
        wordsim_vocab = list()
        wordsim_vocab.extend(wordsim_pairs['word1'].get_values())
        wordsim_vocab.extend(wordsim_pairs['word1'].get_values())
        wordsim_vocab = list(set([w.lower() for w in wordsim_vocab]))
        with open(file_path, 'wb') as doc:
            pickle.dump(wordsim_vocab, doc)
    return wordsim_vocab
  
def read_simverb():
    return pd.read_table('data/SimVerb3500.txt', header=None, usecols=[0, 1, 3]).rename(
        columns={0: 'word1', 1: 'word2', 3: 'gt_sim'})


def extract_simverb_vocabulary(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as doc:
            simverb_vocab = pickle.load(doc)
    else:
        simverb_pairs = read_simverb()
        simverb_vocab = list()
        simverb_vocab.extend(simverb_pairs['word1'].get_values())
        simverb_vocab.extend(simverb_pairs['word2'].get_values())
        simverb_vocab = list(set([w.lower() for w in simverb_vocab]))
        with open(file_path, 'wb') as doc:
            pickle.dump(simverb_vocab, doc)
    return simverb_vocab

def read_simverb():
    return pd.read_table('data/SimVerb3500.txt', header=None, usecols=[0, 1, 3]).rename(
        columns={0: 'word1', 1: 'word2', 3: 'gt_sim'})

In [0]:
simlex_vocab = extract_simlex_vocabulary("/content/drive/My Drive/SimLex999_vocab.pkl")
wordsim_vocab = extract_wordsim_vocabulary("/content/drive/My Drive/WordSim353_vocab.pkl")
simverb_vocab = extract_simverb_vocabulary("/content/drive/My Drive/SimVerb3500_vocab.pkl")

In [0]:
def load_vectors(fname,vocab):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        if tokens[0].lower() in vocab:
            data[tokens[0].lower()] = np.array(tokens[1:],dtype="float")
    return data

In [0]:
simlex_wiki_data = load_vectors("lexvec_enwiki",simlex_vocab);
wordsim_wiki_data = load_vectors("lexvec_enwiki",wordsim_vocab);
simverb_wiki_data = load_vectors("lexvec_enwiki",simverb_vocab);
simlex_crawl_data = load_vectors("lexvec_commoncrawl",simlex_vocab);
wordsim_crawl_data = load_vectors("lexvec_commoncrawl",wordsim_vocab);
simverb_crawl_data = load_vectors("lexvec_commoncrawl",simverb_vocab);

In [0]:
with open('drive/My Drive/LexVec_Wikipedia_WordSim353.pkl', 'wb') as handle:
    pickle.dump(wordsim_wiki_data, handle)
with open('drive/My Drive/LexVec_Wikipedia_SimLex999.pkl', 'wb') as handle:
    pickle.dump(simlex_wiki_data, handle)
with open('drive/My Drive/LexVec_Wikipedia_SimVerb3500.pkl', 'wb') as handle:
    pickle.dump(simverb_wiki_data, handle)
with open('drive/My Drive/LexVec_Crawl_WordSim353.pkl', 'wb') as handle:
    pickle.dump(wordsim_crawl_data, handle)
with open('drive/My Drive/LexVec_Crawl_SimLex999.pkl', 'wb') as handle:
    pickle.dump(simlex_crawl_data, handle)
with open('drive/My Drive/LexVec_Crawl_SimVerb3500.pkl', 'wb') as handle:
    pickle.dump(simverb_crawl_data, handle)

In [21]:
simverb_wiki_data

{'abduct': array([ 0.039729,  0.108977, -0.031411,  0.439261, -0.627875,  0.113092,
         0.073176, -0.293147,  0.06543 , -0.136959,  0.295922, -0.120366,
        -0.310428,  0.186568,  0.061697,  0.244436,  0.010851, -0.218744,
        -0.299025,  0.126442, -0.397199,  0.199806,  0.005745, -0.2403  ,
         0.000881, -0.106256, -0.091025,  0.092012, -0.065712,  0.078694,
        -0.004946, -0.097127, -0.124874,  0.195354,  0.102581, -0.367089,
        -0.058862, -0.165974, -0.236072, -0.013659, -0.190013,  0.202224,
        -0.158917, -0.064565, -0.022191,  0.080454, -0.186262, -0.117164,
         0.191385, -0.329473, -0.072278,  0.14689 ,  0.021658,  0.1441  ,
         0.037596,  0.145936, -0.407853,  0.261022, -0.134992, -0.123671,
         0.482501,  0.113257, -0.13342 , -0.090888, -0.285296,  0.055883,
         0.140238, -0.060193,  0.018986, -0.093607,  0.129274, -0.058295,
         0.147398, -0.024763, -0.349509,  0.095886, -0.10821 , -0.208457,
         0.003601,  0.093594